# Spectral Mesh Flattening

In [230]:
import trimesh
import numpy as np
import scipy
from PIL import Image

## Boundary Functions

In [231]:
def boundary(mesh):
    
    """ A function to find the boundary edges and vertices of a mesh. 
        Inputs:
        mesh: a trimesh mesh. 
        Outputs: 
        boundary_vertices: a list of the indices of vertices on the boundary. 
        next_vertex: a dictionary to find the next vertex on the boundary. 

        Written by: Gabrielle Littlefair
    """

    next_vertex = {}
    boundary_vertices = []
    boundary_edges = []
    start = []

    edges = mesh.edges
    vertex_faces = mesh.vertex_faces
    
    for i in range(len(edges)):

        edge = edges[i]
        v1, v2 = edge
        faces = [j for j in vertex_faces[v1] if j != -1 and j in vertex_faces[v2]]
        # Boundary edges are edges that are only in one face
        if len(faces) == 1:

            boundary_vertices = boundary_vertices + [v1, v2]
            if v1 not in start:
                boundary_edges = boundary_edges + [[v1, v2]]
                next_vertex[v1] = v2
            else:
                boundary_edges = boundary_edges + [[v1, v2]]
                next_vertex[v2] = v1
            
            start = start + boundary_edges[-1][0]
    
    next_v = {}
    for vert in boundary_vertices:

        first_edge, second_edge = [i for i in boundary_edges if vert in i]
        v1 = [i for i in first_edge if i != vert][0]
        v2 = [i for i in second_edge if i != vert][0]
    
        next_v[v1] = vert
        next_v[vert] = v2


    # Remove any duplicates
    boundary_vertices = np.unique(np.array(boundary_vertices))
    
    return boundary_vertices, next_vertex, boundary_edges

def circle_boundary(mesh, centre = [0, 0]):

    """ A function to map the boundary of a mesh to a circle centered at the origin. 
        Inputs:
        mesh: a trimesh mesh. 
        centre: optional argument to change the centre of the circle. 
        
        Outputs: 
        new_boundary_values: an array of boundary values (2 dimensional) on the circle. 

        Written by: Gabrielle Littlefair 
    """

    vertices = mesh.vertices
    radius = 2 # We chose this for best performance with our texture

    # find the boundary information, and intialize the output 
    b_verts, next_vertex, _ = boundary(mesh)
    output = np.zeros((len(vertices), 2))
    weights = {}
    
    # Calculate the "weights" of each edge and store in a dictionary 
    # to be used to determine how far from the other vertices each edge 
    for i in range(len(b_verts)):
        weights[b_verts[i]] = np.linalg.norm(vertices[b_verts[i]] - vertices[next_vertex[b_verts[i]]])
    
    total = sum(weights.values(), 0.0)
    weights = {k: v / total for k, v in weights.items()}

    # Start with one point set at 0 degrees, and then increase the angle each time (since arc length is
    # proportional to angle)
    angle_sum = -np.pi/4
    v1 = b_verts[0]
    output[v1] = [centre[0] + radius * np.cos(angle_sum), centre[1] + radius * np.sin(angle_sum)]

    for i in range(len(b_verts)):
        
        angle = weights[v1] * 2 * np.pi
        angle_sum += angle

        # calculate the new positions
        new_x = centre[0] + radius * np.cos(angle_sum)
        new_y = centre[1] + radius * np.sin(angle_sum)

        v1 = next_vertex[v1]
        output[v1] = [new_x, new_y]

    # make sure the outputs are in the same order as the b_verts 
    new_boundary_values = output[b_verts]

    return new_boundary_values

def get_quartile_length(mesh_vertices, next_vertex_dict):
    """ A function to find 1/4 of the length of all distances around the boundary of a mesh. 
        Inputs:
        mesh_vertices: array of all vertices of a trimesh.
        next_vertex_dict: a dictionary to find the next vertex on the boundary.  

        Outputs:
        quartile_length: 1/4 of the length of total distance around boundary.
        distances: a dictionary to find the distance of a vertex from its previous vertex.

        Written by: Anastasia Anichenko
    """
        
    # get first vertex index on boundary
    current_vertex_idx = list(next_vertex_dict.keys())[0]

    # keep track of distances
    distances = {}
    total_dist = 0.0
    for i in range(len(next_vertex_dict)):
        # get currrent vertex values
        current_vertex = mesh_vertices[current_vertex_idx]

        # get next vertex index
        next_vertex_idx = next_vertex_dict[current_vertex_idx]
        # get next vertex value
        next_vertex = mesh_vertices[next_vertex_dict[current_vertex_idx]]

        dist = np.linalg.norm(next_vertex - current_vertex)
        distances[next_vertex_idx] = dist
        total_dist += dist

        # reset current vertex idk
        current_vertex_idx = next_vertex_idx

    quartile_length = total_dist/4


    return quartile_length, distances

def square_boundary(mesh):

    """ A function to map the boundary of a mesh to a square. 
        Inputs:
        mesh: a trimesh mesh. 
        
        Outputs: 
        new_boundary_values: an array of boundary values (2 dimensional) on the circle. 

        Written by: Anastasia Anichenko
    """

    mesh_vertices = mesh.vertices
    output = np.zeros((len(mesh_vertices), 2))

    unit = 4 # We chose this due to best performance with our texture 

    unit_square = np.array([[0.0,0.0], 
                        [unit,0.0], 
                        [unit,unit],  
                        [0.0,unit]])
    
    boundary_vertices, next_vertex_dict, _ = boundary(mesh)
    quartile_dist, dist_dict = get_quartile_length(mesh_vertices, next_vertex_dict)

    # keep track of total_dist for each edge of square 
    total_dist = 0.0
    # get first vertex index on boundary
    starting_vertex_idx = list(next_vertex_dict.keys())[0]
    current_vertex_idx = starting_vertex_idx
    next_vertex_idx = next_vertex_dict[current_vertex_idx]
    # track vertices to remap (include first two vertices in map)
    vertices_to_map = [current_vertex_idx, next_vertex_idx]
    # include their respective distances to current_vertex
    dist = dist_dict[next_vertex_idx]
    dist_list = [0.0, dist]
    for i in range(4): #loop for each side of square
        # track corners
        A = unit_square[i % len(unit_square)]
        B = unit_square[(i + 1) % len(unit_square)]

        # sum up distances on edge until we reach 1/4 of total distace
        # if final edge of square then sum up all remaining distances (as division will never be perfect)
        while(total_dist + dist < quartile_dist or i == 3):
            next_vertex_idx = next_vertex_dict[current_vertex_idx]

            total_dist += dist
            dist_list.append(total_dist)
            vertices_to_map.append(next_vertex_idx)
            current_vertex_idx = next_vertex_idx
            dist = dist_dict[current_vertex_idx]
            # break once back at starting vertex 
            if(current_vertex_idx == starting_vertex_idx):
                break
        # calculate new positions in two dimensions
        output[vertices_to_map] = [d/total_dist * (B - A) + A for d in dist_list]
        
        # update values for next edge of sqaure
        total_dist = dist
        dist_list = [0.0, dist]
        next_vertex_idx = next_vertex_dict[current_vertex_idx]
        vertices_to_map = [current_vertex_idx, next_vertex_idx]
        current_vertex_idx = next_vertex_idx

    new_boundary_values = output[boundary_vertices]
    return new_boundary_values

## Laplace-Beltrami Functions

In [232]:
def angle(v0, v1, v2):

    """ A function to work out the angle between two edges. 
        Inputs:
        v0: vertex coordinates at the centre of the edges
        v1: vertex of edge1
        v2: vertex of edge2
        
        returns: angle between edge from v0 to v1 and edge from v0 to v2

        Taken from Coursework 2. 
    """

    edge1 = v1 - v0
    edge2 = v2 - v0

    return np.arccos(np.clip(np.dot(edge1, edge2)/(np.linalg.norm(edge1)*np.linalg.norm(edge2)), -1, 1))

def uniformLaplaceBeltrami(mesh):

    """ A function that creates the uniform Laplacian matrix for a given mesh. 
        Inputs:
        mesh: a trimesh mesh
        
        Outputs:
        L: sparse csc matrix containing -1 on the diagonal and 1/k on the off diagonal neighbours.

        Taken from Coursework 2. 
    """

    n = len(mesh.vertices)
    L = scipy.sparse.lil_matrix(-1*np.eye(n))
    neighbours = mesh.vertex_neighbors

    for i in range(n):
        L[i, neighbours[i]] = 1/len(neighbours[i])

    return L.tocsr()

def cotanLaplaceBeltrami(mesh):

    """ A function to find the cotangent discretization of the Laplacian. 
        Inputs:
        mesh: a trimesh mesh
        Outputs:
        L: the cotangent discretization of the Laplacian. 

        Taken from Coursework 2. 
    """

    vs = mesh.vertices
    fs = mesh.faces
    vert_ns = mesh.vertex_neighbors
    vert_fs = mesh.vertex_faces

    n = len(vs)
    areas = np.zeros(n)
    C = scipy.sparse.lil_matrix(np.zeros((n, n)))
    
    for vertex in range(n):

        # for every vertex find the neighbours and the faces
        faces = vert_fs[vertex][vert_fs[vertex] != -1]

        for neighbour in vert_ns[vertex]:
            # for every neighbour find the faces that are shared with the vertex, and the other vertices on those faces
            faces = [i for i in vert_fs[vertex] if neighbour in fs[i] and i != -1]
            vertices = [i for i in fs[faces].flatten() if not i in [vertex, neighbour, -1]]

            if len(vertices) > 1:
                
                angle1 = angle(vs[vertices[0]], vs[vertex], vs[neighbour])
                angle2 = angle(vs[vertices[1]], vs[vertex], vs[neighbour])
            
            # This is the case where the mesh is not closed
            else:
                angle1 = angle(vs[vertices[0]], vs[vertex], vs[neighbour])
                angle2 = angle1

            # account for division by zero (and near zero)
            if abs(np.tan(angle1)) < 1e-10:
                cot_alpha = 1e10
            else:
                cot_alpha = 1/np.tan(angle1)

            if abs(np.tan(angle2)) < 1e-10:
                cot_beta = 1e10
            else:
                cot_beta = 1/np.tan(angle2)
            
            # Assign the values 
            C[vertex, neighbour] = cot_alpha + cot_beta
            C[vertex, vertex] -= cot_alpha + cot_beta 

            # voronoi areas 
            areas[vertex] += (C[vertex, neighbour] * np.linalg.norm(vs[vertex] - vs[neighbour])**2) / 8

    M_inv = scipy.sparse.diags(1/(2*areas))
    
    L = (M_inv @ C)
    return L.tocsr()

def mvLaplaceBeltrami(mesh):

    """ A function to find the mean value weighted discretization of the Laplace Beltrami.
        Inputs:
        mesh: a trimesh mesh
        Outputs:
        L: the mean value discretization of the Laplacian.

        Written by: Gabrielle Littlefair
    """

    vs = mesh.vertices
    fs = mesh.faces
    vert_ns = mesh.vertex_neighbors
    vert_fs = mesh.vertex_faces

    n = len(vs)
    L = scipy.sparse.lil_matrix(np.zeros((n, n)))

    for vertex in range(n):
        # for every vertex find the neighbours and the faces 
        faces = vert_fs[vertex][vert_fs[vertex] != -1]

        for neighbour in vert_ns[vertex]:
            # for every neighbour find the faces that are shared with the vertex, and the other vertices on those faces
            faces = [i for i in vert_fs[vertex] if neighbour in fs[i] and i != -1] 
            vertices = [i for i in fs[faces].flatten() if not i in [vertex, neighbour, -1]]

            if len(vertices) > 1:

                angle1 = angle(vs[vertex], vs[vertices[0]], vs[neighbour])/2
                angle2 = angle(vs[vertex], vs[vertices[1]], vs[neighbour])/2
            
            # This is the case where the mesh is not closed
            else:
                angle1 = angle(vs[vertex], vs[vertices[0]], vs[neighbour])/2
                angle2 = angle1/2
            
            # Assign the values 
            L[vertex, neighbour] = (np.tan(angle1) + np.tan(angle2))/np.linalg.norm(vs[vertex] - vs[neighbour])
            L[vertex, vertex] -= (np.tan(angle1) + np.tan(angle2))/np.linalg.norm(vs[vertex] - vs[neighbour])

    return L.tocsr() 

def improvedLaplaceBeltrami(mesh):

    """ A function to find the improved Laplacian (weighted by number of neighbours). 
        Inputs:
        mesh: a trimesh mesh
        Outputs:
        L: the improved discretization of the Laplacian.

        Written by: Anastasia Anichenko 
    """

    vs = mesh.vertices
    vert_ns = mesh.vertex_neighbors

    n = len(vs)
    W = np.zeros((n, n))
    
    # populate W matrix which weights by number of neighbours
    for vertex in range(n):
        for neighbour in vert_ns[vertex]:
            dist = np.linalg.norm(vs[vertex] - vs[neighbour])**2
            W[vertex, neighbour] = 1/len(vert_ns[vertex]) * np.exp(-dist / 4)

    # compute D matrix
    D = scipy.sparse.diags(np.asarray(W.sum(axis=1)), 0, format='csr')
    # compute L matrix
    L = scipy.sparse.lil_matrix(W - D)

    return L.tocsr()

## Embedding Functions

In [233]:
def tutte_embedding(mesh, boundary_function = circle_boundary, LaplaceBeltrami = cotanLaplaceBeltrami):

    """ A function that uses Tutte's embedding method to flatten a mesh
        with given boundary conditions. 
        Inputs: 
        mesh: a trimesh mesh. 
        boundary_function: a function that returns the 2D coordinates for the boundary vertices.
         
        Outputs: 
        flat_mesh: the flattened mesh. 

        Written by: Gabrielle Littlefair. 
    """

    flat_mesh = mesh.copy()

    vertices = mesh.vertices
    b_verts, _, _ = boundary(mesh)
    b_vals = boundary_function(mesh)
    L = LaplaceBeltrami(mesh)
    L = L.tolil()
    
    # Change the value of the laplace beltrami for the boundary vertices 
    # this is adding in the constraints
    for vert in b_verts:
        L[vert, :] = np.zeros(L.shape[1])
        L[vert, vert] = 1
    
    L = L.tocsc()
    
    # set up the zero vector
    B = np.zeros((len(vertices), 2))
    # add the constraint values 
    B[b_verts] = b_vals

    X = scipy.sparse.linalg.spsolve(L, B)
    flat_mesh.vertices = np.hstack((X, np.zeros((X.shape[0], 1))))
    return flat_mesh

def free_boundary(mesh, LaplaceBeltrami = cotanLaplaceBeltrami):

    """ A function that uses Tutte's embedding method to flatten a mesh
        with free boundaries. 
        Inputs: 
        mesh: a trimesh mesh. 
        LaplaceBeltrami: function to calculate the Laplace Beltrami. 
         
        Outputs: 
        flat_mesh: the flattened mesh. 

        Written by: Gabrielle Littlefair. 
    """

    r = 8 # we chose this for best performance with our texture

    flat_mesh = mesh.copy()
    L = LaplaceBeltrami(mesh)

    _, v = scipy.sparse.linalg.eigs(L, 3, which='SM')

    X = r**2 * np.real(v[:, 1:])/np.linalg.norm(v[:, 1:], axis = 0)
    flat_mesh.vertices = np.hstack((X, np.zeros((X.shape[0], 1))))

    return flat_mesh


def export_textured(mesh_filename, export_name, flat_export_name, bound_func=circle_boundary, texture_file='textures/CheckTexture.png', LB=cotanLaplaceBeltrami):

    """ A function to export a mesh and a flattened mesh with a texture using the 
        flat mesh as the UV parametrization. 
        
        Inputs:
        mesh_filename: mesh to import. 
        export_name: name that the original mesh (with texture) will be saved to. 
        flat_export_name: flattened textured mesh export name. 
        texture_name: optional argument to determine which texture to apply. 

        Written by: Gabrielle Littlefair
        
    """

    mesh = trimesh.load(mesh_filename)
    n = len(mesh.vertices)

    flat_mesh = tutte_embedding(mesh, boundary_function=bound_func, LaplaceBeltrami=LB)
    uv_coordinates = flat_mesh.vertices[:, :2]
    
    texture = Image.open(texture_file)

    mesh.visual = trimesh.visual.TextureVisuals(uv=uv_coordinates)
    flat_mesh.visual = trimesh.visual.TextureVisuals(uv=uv_coordinates)

    mesh.visual.material.image = texture
    flat_mesh.visual.material.image = texture

    mesh.export(export_name, include_texture=True)
    flat_mesh.export(flat_export_name, include_texture=True)

    return 

def export_textured_free(mesh_filename, export_name, flat_export_name, texture_file='textures/CheckTexture.png', LB=cotanLaplaceBeltrami):

    """ A function to export a mesh and a flattened mesh (with a free boundary) with a texture using the 
        flat mesh as the UV parametrization. 
        
        Inputs:
        mesh_filename: mesh to import. 
        export_name: name that the original mesh (with texture) will be saved to. 
        flat_export_name: flattened textured mesh export name. 
        texture_name: optional argument to determine which texture to apply. 

        Written by: Gabrielle Littlefair
        
    """

    mesh = trimesh.load(mesh_filename)
    flat_mesh = free_boundary(mesh, LaplaceBeltrami = LB)
    uv_coordinates = flat_mesh.vertices[:, :2]
    texture = Image.open(texture_file)

    mesh.visual = trimesh.visual.TextureVisuals(uv=uv_coordinates)
    flat_mesh.visual = trimesh.visual.TextureVisuals(uv=uv_coordinates)

    mesh.visual.material.image = texture
    flat_mesh.visual.material.image = texture

    mesh.export(export_name, include_texture=True)
    flat_mesh.export(flat_export_name, include_texture=True)

    return 

## Seam Functions

In [211]:
import networkx as nx
import matplotlib as mpl
import matplotlib.cm as cm

def curvature_weights(mesh, length_divisor = 1.1):

    """ Function to compute the weights for each edge (based on Gaussian curvature),
        to be used in algorithms finding the shortest (or cheapest) path. 
        Inputs:
        mesh: a trimesh mesh. 
        length_divisor: optional parameter that toggles how much the length of each edge
                        is used in the weight. 
        
        Outputs: 
        ws: non-negative weight for each edge in the mesh.

        Written by: Gabrielle Littlefair 
    """

    edges = mesh.edges_unique
    length = mesh.edges_unique_length
    # find Gaussian curvature
    gc = trimesh.curvature.discrete_gaussian_curvature_measure(mesh, mesh.vertices, 1)

    ws = np.zeros(len(edges))
    for edge in range(len(edges)):
        # use difference in gaussian curvature, so that moving towards higher curvature is prioritized
        ws[edge] = gc[edges[edge][1]] #gc[edges[edge][0]] - gc[edges[edge][1]]

    # scale the length to whatever proportion (for weighting)
    length = ws.max()/length_divisor * (length - length.min())/(length.max() - length.min())
    # add the curvature weights and the scaled length 
    ws = ws + length
    
    return np.where(ws < 0, 0, ws)

def waypoints(mesh, percent = 2, threshold = 1):

    """ A function to find points (in order) that the seam of the mesh should go through. 
        This function will find high curvature points, and then create a graph from them, 
        as well as a minimal spanning tree, and then finally a series of edges that define
        which vertices to find shortest distance between in order. 

        Inputs:
        mesh: a trimesh mesh. 
        percent: what percentage of the mesh to keep as points. 
        threshold: threshold to be used to determine if points should be removed if they are within
                   the threshold of each other (in distance). 
        
        Outputs: 
        tree.edges: list of edges that define start and end points of paths to be found using dijkstra's algorithm
                    in seam(). 

        Written by: Gabrielle Littlefair
    
    """

    verts = mesh.vertices
    gc = trimesh.curvature.discrete_gaussian_curvature_measure(mesh, mesh.vertices, 1)
    indices = np.where(gc > np.percentile(gc, 100 - percent))[0].tolist()
    # Only keep points that are not close too together
    indices_copy = indices.copy()
    for i in indices_copy:
        distances = [np.linalg.norm(verts[i] - verts[j]) for j in indices if j != i]
        if np.any(np.array(distances) < threshold):
                indices.remove(i)

    # Create graph of necessary points
    g = nx.Graph()
    for i in indices:
        count = 0
        for j in indices:
            if j == i:
                continue
            if count == 0:
                g.add_edge(i, j, weight=np.linalg.norm(verts[i] - verts[j]))
            else:
                g.add_edge(j, i, weight=np.linalg.norm(verts[i] - verts[j]))
            count += 1

    # find minimal spanning tree. 
    tree = nx.minimum_spanning_tree(G = g, weight ='weight')
    return tree.edges


def seam(mesh, threshold = 1, percent = 2, length_divisor = 1.1):

    """ A function to find the seam along which to cut a closed mesh to create an open mesh. 
        Inputs:
        mesh: a trimesh mesh. 
        threshold: used to determine radius of non-maximal suppression. 
        percent: what percent of the mesh vertices to use as necessary points. 
        length_divisor: factor to determine how much weight the length each edge should account for. 
        
        Outputs: 
        final_path: list of lists of vertices to travel through. 

        Written by: Gabrielle Littlefair 
    """

    paths = waypoints(mesh, percent = percent, threshold = threshold)
    edges = mesh.edges_unique
    weights = curvature_weights(mesh, length_divisor)


    final_path = []

    g = nx.Graph()
    for edge, w in zip(edges, weights):
         g.add_edge(*edge, weight=w)

    for edge in paths:
        start, stop = edge
        path = nx.dijkstra_path(g, source=start, target=stop, weight = 'weight')
        final_path += [path]
    
    val = False
    if len(final_path) == 1:
        val == True

    # This merges routes that end or start with the same vertices. 
    while not val:
        testing = np.unique([[k[0], k[-1]] for k in final_path])
        val = len(testing) == 2*len(final_path)

        path = final_path[0]
        next_path = [j for j in final_path if j[0] == path[-1]]
        next_path_backup = [j[::-1] for j in final_path if j[-1] == path[-1] and j != path]

        if len(next_path) == 0:

            if len(next_path_backup) == 0:
                final_path.remove(path)
                final_path.append(path[::-1])
            elif len(next_path_backup) in [1, 2]:
                final_path.remove(next_path_backup[0][::-1])
                final_path[0] += next_path_backup[0][1:]

        elif len(next_path) in [1, 2]:
            final_path.remove(next_path[0])
            final_path[0] += next_path[0][1:]

    return final_path

def delete_faces_along_seam(mesh, threshold = 1, percent = 2, length_divisor = 1.1):
    """ A function to delete all faces along a seam on a closed mesh. 

        Inputs: 
        mesh: a trimesh mesh
        threshold: used to determine radius of non-maximal suppression. 
        percent: what percent of the mesh vertices to use as necessary points. 
        length_divisor: factor to determine how much weight the length each edge should account for. 
        
        Outputs: 
        mesh_seam_del: a mesh with faces deleted along seam

        Written by: Anastasia Anichenko 
        
    """

    faces = mesh.faces
    vertex_faces = mesh.vertex_faces

    seam_paths = seam(mesh, threshold=threshold, percent=percent, length_divisor=length_divisor)

    face_indices_to_delete = []
    # loop over all paths and append faces neighboring seam
    for path in (seam_paths):
        for i in range(len(path)):
            vertex_idx = path[i]
            vertex_faces_i = vertex_faces[vertex_idx]
            vertex_face_indices = vertex_faces_i[np.where(vertex_faces_i.flatten() != -1)] #-1 is filler so filter out
            
            face_indices_to_delete.extend(vertex_face_indices)
    
    mesh_seam_del = mesh.copy()
    mesh_seam_del.faces = np.delete(faces, face_indices_to_delete, axis=0)

    return mesh_seam_del


def virtual_vertices(mesh):

    """ A function to add virtual vertices to non-closed meshes.
        This function only works with one open section in a mesh. 

        This functions works by using the boundary edges, and reflecting the vertex that connects
        to the two vertices in the boundary edge across the edge. This then created a second face for
        the edge, and then I connect all of these reflected vertices to each other. 

        Inputs: 
        mesh: a trimesh mesh
        Outputs: a mesh with a virtual border added. 

        Written by: Gabrielle Littlefair
        
    """

    # Create mesh copy and find the boundary vertices and boundary edges
    extended_mesh = mesh.copy()
    boundary_verts, next_vertex, boundary_edges = boundary(mesh)
    vs = mesh.vertices
    neighbs = mesh.vertex_neighbors
    fs = mesh.faces
    normals = mesh.vertex_normals
    n = len(vs)
    virt_faces = [] # this will be a list of virtual faces to add
    virt_verts = [] # this will be a list of virtual vertices to add
    virt_vert_faces = [[i] for i in boundary_verts] # this will be the virtual faces that join only virtual vertices
    
    for edge in boundary_edges:

        index = [i for i in neighbs[edge[0]] if i in neighbs[edge[1]]][0]
        point = vs[index]
        
        # for every boundary face, reflect the inner vertex across the boundary edge
        A_point = point - vs[edge[0]]
        AB = (vs[edge[1]] - vs[edge[0]])/np.linalg.norm((vs[edge[1]] - vs[edge[0]]))
        t = np.dot(A_point, AB)
        projected_point = vs[edge[0]] + t*AB
        projected_point = 2*projected_point - point

        # # create new faces with the new vertices
        indices = [np.where(boundary_verts == edge[0])[0][0], np.where(boundary_verts == edge[1])[0][0]]

        if len(virt_vert_faces[indices[0]]) < 3: 
            virt_vert_faces[indices[0]] += [n + len(virt_verts)]
        if len(virt_vert_faces[indices[1]]) < 3:
            virt_vert_faces[indices[1]] += [n + len(virt_verts)]

        if next_vertex[edge[0]] != edge[1]:
            virt_faces += [[edge[0], edge[1], n + len(virt_verts)]]
        else:
            virt_faces += [[edge[1], edge[0], n + len(virt_verts)]]
            
        virt_verts += [projected_point]
        
        true_normal = (normals[edge[0]] +  normals[edge[1]] + normals[index])/3
        for i in indices:
            if len(virt_vert_faces[i]) == 3:
                v1, v2, v3 = virt_vert_faces[i]
                normal1 = np.cross((virt_verts[v2 - n] - vs[v1]), (virt_verts[v3 - n] - virt_verts[v2 - n]))
                if np.dot(normal1, true_normal) < 0:
                    virt_vert_faces[i] = [v2, v1, v3]
        
    virt_faces = np.array(virt_faces)
    extended_mesh.vertices = np.vstack((vs, virt_verts))
    extended_mesh.faces = np.vstack((fs, virt_faces, virt_vert_faces))

    return extended_mesh

## Work-In-Progress Functions 

In [178]:
def reconstruct_faces(original_mesh, mesh_faces_removed, seam_vertex_indices):

    """ WIP A function to reconstruct faces that have been deleted 
        along the seam using virtual vertices. For the most part the reconstruction works well,
        struggles in certain areas of bigger holes. 

        This functions is based on the virtual_vertices function defined earlier in the notebook.

        Inputs: 
        original_mesh: original mesh prior to removing faces along seam
        mesh_faces_removed: mesh with faces removed along the seam
        seam_vertex_indices: vertex indices of the seam
        Outputs: 
        extended_mesh: a mesh with the faces reconstrucred so that there is a cut along the seam. 

        Written by: Anastasia Anichenko
        
    """

    # Create mesh copy and find the boundary vertices and boundary edges
    extended_mesh = mesh_faces_removed.copy()
    boundary_verts, next_vertex, boundary_edges = boundary(mesh_faces_removed)
    vs = mesh_faces_removed.vertices
    original_vs = original_mesh.vertices
    neighbs = mesh_faces_removed.vertex_neighbors
    original_neighbs = original_mesh.vertex_neighbors
    fs = mesh_faces_removed.faces
    normals = mesh_faces_removed.vertex_normals
    n = len(vs)
    virt_faces = [] # this will be a list of virtual faces to add
    virt_verts = [] # this will be a list of virtual vertices to add
    virt_vert_faces = [[i] for i in boundary_verts] # this will be the virtual faces that join only virtual vertices
    vertex_order = []
    
    for edge in boundary_edges:

        index = [i for i in neighbs[edge[0]] if i in neighbs[edge[1]]][0]
        
        # create new faces with the new vertices
        indices = [np.where(boundary_verts == edge[0])[0][0], np.where(boundary_verts == edge[1])[0][0]]

        if len(virt_vert_faces[indices[0]]) < 3:
            virt_vert_faces[indices[0]] += [n + len(virt_verts)]
        if len(virt_vert_faces[indices[1]]) < 3:   
            virt_vert_faces[indices[1]] += [n + len(virt_verts)]

        if next_vertex[edge[0]] != edge[1]:
            virt_faces += [[edge[0], edge[1], n + len(virt_verts)]]
        else:
            virt_faces += [[edge[1], edge[0], n + len(virt_verts)]]
            
        # create the "virtual" vertices on the seam
        edge_neighbors = [vertex for vertex in original_neighbs[edge[0]] if vertex in original_neighbs[edge[1]]]
        seam_vertex_idx = [vertex for vertex in edge_neighbors if vertex in seam_vertex_indices][0]
        virt_verts.append(vs[seam_vertex_idx])
        vertex_order.append(seam_vertex_idx) # keep track of order

        true_normal = (normals[edge[0]] +  normals[edge[1]] + normals[index])/3
        for i in indices:
            if len(virt_vert_faces[i]) == 3:
                v1, v2, v3 = virt_vert_faces[i]
                normal1 = np.cross((virt_verts[v2 - n] - vs[v1]), (virt_verts[v3 - n] - virt_verts[v2 - n]))
                if np.dot(normal1, true_normal) < 0:
                    virt_vert_faces[i] = [v2, v1, v3]
        
    virt_faces = np.array(virt_faces)
    extended_mesh.vertices = np.vstack((vs, virt_verts))
    extended_mesh.faces = np.vstack((fs, virt_faces, virt_vert_faces))

    return extended_mesh, vertex_order

def split_seam(mesh, seam_paths):
    """ WIP A function to split mesh along a seam. 

        Inputs: 
        mesh: a trimesh mesh
        seam_paths: a list of paths to define the seam
        Outputs: 
        mesh_with_cut: a mesh with a cut along the defined seam
        cut_vertex_order: order of the newly defined vertices along the seam

        Written by: Anastasia Anichenko
        
    """
    mesh_seam_del = delete_faces_along_seam(mesh, seam_paths)
    seam_paths_flattend = list(set([vertex for path in seam_paths for vertex in path]))
    mesh_with_cut, cut_vertex_order = reconstruct_faces(mesh, mesh_seam_del, seam_paths_flattend)

    return mesh_with_cut, cut_vertex_order

In [229]:
# Results for seam 
hippo = trimesh.load("closed_meshes/hippo.obj")

hippo_points = waypoints(hippo, threshold=0.4)
hippo_points = [j for i in hippo_points for j in i]
hippo_p_mesh = hippo.copy()
hippo_p_mesh.visual.vertex_colors[hippo_points] = [255, 0, 0, 1]
hippo_p_mesh.export("g_meshes/points_hippo.obj")

hippo_seam = seam(hippo, threshold=0.2, length_divisor = 0.01, percent = 2)
hippo_verts = [j for i in hippo_seam for j in i]
colour_hippo = hippo.copy()
colour_hippo.visual.vertex_colors[hippo_verts] = [242, 255, 94, 1]
colour_hippo.visual.vertex_colors[hippo_points] = [255, 0, 0, 1]
colour_hippo.export("g_meshes/colour_hippo.obj")

# # visualise seam for each 
s1 = delete_faces_along_seam(hippo, length_divisor = 0.01, percent = 2, threshold=0.4)
s1.export("g_meshes/hippo_seam.obj")

'# https://github.com/mikedh/trimesh\nv -4.32719990 -0.19460000 -4.41720010\nv -4.32450010 -0.18360000 -4.44439980\nv -4.27430010 -0.19840000 -4.44380000\nv -4.27750020 -0.21340000 -4.41610000\nv -4.23140000 -0.21950000 -4.44490000\nv -4.23269990 -0.23390000 -4.41309980\nv -4.28240010 -0.23400000 -4.39069990\nv -4.24240020 -0.25960000 -4.38670020\nv -4.33010010 -0.21270000 -4.39239980\nv -4.21040010 -0.26230000 -4.45060010\nv -4.21220020 -0.27790000 -4.41919990\nv -4.21080020 -0.32060000 -4.46040010\nv -4.21589990 -0.33820000 -4.43249990\nv -4.22160010 -0.30290000 -4.39160010\nv -4.21320010 -0.35650000 -4.40150020\nv -4.25990010 -0.28720000 -4.36469980\nv -4.23560000 -0.32840000 -4.36550000\nv -4.21140000 -0.37350000 -4.36579990\nv -4.26070020 -0.35180000 -4.34100010\nv -4.23250010 -0.39750000 -4.33909990\nv -4.28900000 -0.31060000 -4.34550000\nv -4.33699990 -0.23430000 -4.37200020\nv -4.29449990 -0.25710000 -4.36910010\nv -4.31920000 -0.27550000 -4.35160020\nv -4.34509990 -0.25140000 

In [226]:
## Experiment 1, Circle Boundary vs Square Boundary vs Free Boundary 

face = trimesh.load("open_meshes/Face.obj")
horse = trimesh.load("open_meshes/Horse2_smaller.obj")

circle1 = tutte_embedding(face)
square1 = tutte_embedding(face, boundary_function = square_boundary)
free1 = free_boundary(face)

circle1.export("g_meshes/E1/circle1.obj")
square1.export("g_meshes/E1/square1.obj")
free1.export("g_meshes/E1/free1.obj")

circle2 = tutte_embedding(face)
square2 = tutte_embedding(face, boundary_function = square_boundary)
free2 = free_boundary(face)

circle2.export("g_meshes/E1/circle2.obj")
square2.export("g_meshes/E1/square2.obj")
free2.export("g_meshes/E1/free2.obj")

## With texture 

export_textured("open_meshes/Face.obj", "g_meshes/E1/E1_tex1.obj", "g_meshes/E1/flat1.obj", bound_func = circle_boundary)
export_textured("open_meshes/Face.obj", "g_meshes/E1/E1_tex2.obj", "g_meshes/E1/flat2.obj", bound_func = square_boundary)
export_textured_free("open_meshes/Face.obj", "g_meshes/E1/E1_tex3.obj", "g_meshes/E1/flat3.obj")

export_textured("open_meshes/Horse2_smaller.obj", "g_meshes/E1/E1_tex4.obj", "g_meshes/flat1.obj", bound_func = circle_boundary)
export_textured("open_meshes/Horse2_smaller.obj", "g_meshes/E1/E1_tex5.obj", "g_meshes/flat2.obj", bound_func = square_boundary)
#export_textured_free("open_meshes/Horse2_smaller.obj", "g_meshes/E1/E1_tex6.obj", "g_meshes/flat3.obj")

KeyboardInterrupt: 

In [225]:
## Experiment 2, Laplace Beltrami Disretizations

export_textured("open_meshes/Face.obj", "g_meshes/E2/uniform1.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = uniformLaplaceBeltrami)
export_textured("open_meshes/Face.obj", "g_meshes/E2/cotan1.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = cotanLaplaceBeltrami)
export_textured("open_meshes/Face.obj", "g_meshes/E2/mv1.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = mvLaplaceBeltrami)
export_textured("open_meshes/Face.obj", "g_meshes/E2/imp1.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = improvedLaplaceBeltrami)

export_textured("open_meshes/Horse2_small.obj", "g_meshes/E2/uniform2.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = uniformLaplaceBeltrami)
export_textured("open_meshes/Horse2_small.obj", "g_meshes/E2/cotan2.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = cotanLaplaceBeltrami)
export_textured("open_meshes/Horse2_small.obj", "g_meshes/E2/mv2.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = mvLaplaceBeltrami)
export_textured("open_meshes/Horse2_small.obj", "g_meshes/E2/imp2.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = improvedLaplaceBeltrami)

export_textured("open_meshes/ear.obj", "g_meshes/E2/uniform3.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = uniformLaplaceBeltrami)
export_textured("open_meshes/ear.obj", "g_meshes/E2/cotan3.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = cotanLaplaceBeltrami)
export_textured("open_meshes/ear.obj", "g_meshes/E2/mv3.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = mvLaplaceBeltrami)
export_textured("open_meshes/ear.obj", "g_meshes/E2/imp3.obj", "g_meshes/flat1.obj", bound_func = circle_boundary, LB = improvedLaplaceBeltrami)

In [108]:
## Experiment 3, Large vs Small meshes

big_horse = trimesh.load("open_meshes/Horse2.obj")
middle_horse = trimesh.load("open_meshes/Horse2_small.obj")
small_horse = trimesh.load("open_meshes/Horse2_smaller.obj")

print(f"The biggest mesh has {len(big_horse.vertices)} vertices, {len(big_horse.edges)} edges, and {len(big_horse.faces)} faces.")
print(f"The middle mesh has {len(middle_horse.vertices)} vertices, {len(middle_horse.edges)} edges, and {len(middle_horse.faces)} faces.")
print(f"The smallest mesh has {len(small_horse.vertices)} vertices, {len(small_horse.edges)} edges, and {len(small_horse.faces)} faces.")

export_textured("open_meshes/Horse2.obj", "g_meshes/E3/big1.obj", "g_meshes/flat1.obj")
export_textured("open_meshes/Horse2_small.obj", "g_meshes/E3/medium1.obj", "g_meshes/flat1.obj")
export_textured("open_meshes/Horse2_smaller.obj", "g_meshes/E3/small1.obj", "g_meshes/flat1.obj")

export_textured("open_meshes/Horse2.obj", "g_meshes/E3/big2.obj", "g_meshes/flat1.obj", bound_func= square_boundary)
export_textured("open_meshes/Horse2_small.obj", "g_meshes/E3/medium2.obj", "g_meshes/flat1.obj", bound_func= square_boundary)
export_textured("open_meshes/Horse2_smaller.obj", "g_meshes/E3/small2.obj", "g_meshes/flat1.obj", bound_func = square_boundary)

The biggest mesh has 38019 vertices, 227592 edges, and 75864 faces.
The middle mesh has 9557 vertices, 56895 edges, and 18965 faces.
The smallest mesh has 4805 vertices, 28443 edges, and 9481 faces.


In [117]:
## Experiment 4 how long the mesh is 

export_textured("open_meshes/Horse1.obj", "g_meshes/E4/Horse1.obj", "g_meshes/E4/t1.obj")
export_textured("open_meshes/Horse2.obj", "g_meshes/E4/Horse2.obj", "g_meshes/E4/t2.obj")
export_textured("open_meshes/Horse3.obj", "g_meshes/E4/Horse3.obj", "g_meshes/E4/t3.obj")
export_textured("open_meshes/Horse4.obj", "g_meshes/E4/Horse4.obj", "g_meshes/E4/t4.obj")
export_textured("open_meshes/Horse5.obj", "g_meshes/E4/Horse5.obj", "g_meshes/E4/t5.obj")

horse1 = trimesh.load("open_meshes/Horse1.obj")
horse2 = trimesh.load("open_meshes/Horse2.obj")
horse3 = trimesh.load("open_meshes/Horse3.obj")
horse4 = trimesh.load("open_meshes/Horse4.obj")
horse5 = trimesh.load("open_meshes/Horse5.obj")

flat1 = tutte_embedding(horse1)
flat2 = tutte_embedding(horse2)
flat3 = tutte_embedding(horse3)
flat4 = tutte_embedding(horse4)
flat5 = tutte_embedding(horse5)

flat1.export("g_meshes/E4/flat1.obj")
flat2.export("g_meshes/E4/flat2.obj")
flat3.export("g_meshes/E4/flat3.obj")
flat4.export("g_meshes/E4/flat4.obj")
flat5.export("g_meshes/E4/flat5.obj")

'# https://github.com/mikedh/trimesh\nv 0.19177950 0.53742274 0.00000000\nv 0.18630042 0.53719521 0.00000000\nv 0.18697629 0.53096299 0.00000000\nv 0.19418327 0.53342658 0.00000000\nv 0.17806904 0.53701527 0.00000000\nv 0.17819007 0.52970546 0.00000000\nv 0.17894460 0.52209849 0.00000000\nv 0.18867577 0.52435080 0.00000000\nv 0.19696488 0.52769627 0.00000000\nv 0.15616365 0.53726249 0.00000000\nv 0.15573292 0.52847351 0.00000000\nv 0.16773264 0.52891565 0.00000000\nv 0.16793859 0.53702162 0.00000000\nv 0.16788797 0.52055792 0.00000000\nv 0.15549447 0.51948415 0.00000000\nv 0.16912421 0.50197701 0.00000000\nv 0.16837066 0.51165805 0.00000000\nv 0.15547749 0.51000115 0.00000000\nv 0.15570986 0.49984502 0.00000000\nv 0.18141493 0.50469951 0.00000000\nv 0.18004391 0.51383759 0.00000000\nv 0.20236461 0.51263361 0.00000000\nv 0.19964109 0.52075875 0.00000000\nv 0.19053063 0.51679802 0.00000000\nv 0.19254988 0.50819257 0.00000000\nv 0.19454108 0.54168105 0.00000000\nv 0.18752603 0.54357460 0.

In [119]:
## Experiment 5 Nice Triangulation vs Worse Triangulation

bad_horse = trimesh.load("open_meshes/Horse2_smaller.obj")
nice_horse = trimesh.load("open_meshes/Horse2_smaller_remeshed.obj")

export_textured("open_meshes/Horse2_smaller.obj", "g_meshes/E5/bad_horse.obj", "g_meshes/E5/t1.obj")
export_textured("open_meshes/Horse2_smaller_remeshed.obj", "g_meshes/E5/good_horse.obj", "g_meshes/E5/t2.obj")

flat1 = tutte_embedding(bad_horse)
flat2 = tutte_embedding(nice_horse)

flat1.export("g_meshes/E5/flat1.obj")
flat2.export("g_meshes/E5/flat2.obj")

'# https://github.com/mikedh/trimesh\nv -0.43235236 -0.68362676 0.00000000\nv -0.43322851 -0.68321974 0.00000000\nv -0.43377956 -0.68339967 0.00000000\nv 0.09205275 -0.18837972 0.00000000\nv -0.12302049 -1.95929205 0.00000000\nv 0.10066937 -1.91113824 0.00000000\nv 0.15607559 -1.80572023 0.00000000\nv 0.21031482 -0.13953154 0.00000000\nv 0.17145036 -0.12346737 0.00000000\nv 0.22806672 -0.12132868 0.00000000\nv -0.41314670 -0.88405802 0.00000000\nv -0.42870136 -0.85637829 0.00000000\nv -0.40935267 -0.90543785 0.00000000\nv -0.41511302 -0.86482428 0.00000000\nv -0.39180710 -0.88730503 0.00000000\nv -0.35620776 -0.88077690 0.00000000\nv -0.37142232 -0.89396413 0.00000000\nv -0.37061372 -1.07459310 0.00000000\nv -0.35440803 -1.04837282 0.00000000\nv -0.33287412 -1.12937067 0.00000000\nv -0.33571155 -1.07460559 0.00000000\nv -0.32582750 -1.04864140 0.00000000\nv -0.40186072 -0.66412024 0.00000000\nv -0.41079925 -0.66436345 0.00000000\nv -0.41073413 -0.66954005 0.00000000\nv -0.41260249 -0.6

In [206]:
# Experiment 6 / Not really an experiment just what different meshes look like
bcube = trimesh.load("closed_meshes/bumpy-cube.obj")
hippo = trimesh.load("closed_meshes/Hippo.obj")
horse = trimesh.load("closed_meshes/horse_60000.obj")

horse_points = waypoints(horse, threshold=2.8)
horse_points = [j for i in horse_points for j in i]
horse_p_mesh = horse.copy()
horse_p_mesh.visual.vertex_colors[horse_points] = [255, 0, 0, 1]
horse_p_mesh.export("g_meshes/points_horse.obj")

horse_seam = seam(horse, threshold=2.8, length_divisor = 0.01)
horse_verts = [j for i in horse_seam for j in i]
colour_horse = horse.copy()
colour_horse.visual.vertex_colors[horse_verts] = [242, 255, 94, 1]
colour_horse.export("g_meshes/colour_horse.obj")

colour_horse.show()

# # visualise seam for each 

s1 = delete_faces_along_seam(bcube, length_divisor = 0.01)
s2 = delete_faces_along_seam(hippo, length_divisor = 0.01)
s3 = delete_faces_along_seam(horse, length_divisor = 0.01, threshold=2.8)

s1.export("g_meshes/E6/s_bcube.obj")
s2.export("g_meshes/E6/s_hippo.obj")
s3.export("g_meshes/E6/s_horse.obj")

export_textured("g_meshes/E6/s_bcube.obj", "g_meshes/E6/tex_bcube.obj", "g_meshes/flat1.obj")
export_textured("g_meshes/E6/s_hippo.obj", "g_meshes/E6/tex_hippo.obj", "g_meshes/flat1.obj")
export_textured("g_meshes/E6/s_horse.obj", "g_meshes/E6/tex_horse.obj", "g_meshes/flat1.obj")

KeyboardInterrupt: 

In [207]:
# Experiment 7 

# Using different levels of gaussian curvature weighting 
hippo = trimesh.load("closed_meshes/hippo.obj")

seam1 = seam(hippo, length_divisor = 0.01, threshold = 0.2)
seam2 = seam(hippo, length_divisor = 0.5, threshold = 0.2)
seam3 = seam(hippo, length_divisor = 1, threshold = 0.2)
seam4 = seam(hippo, length_divisor = 1.5, threshold = 0.2)

seam1 = [j for i in seam1 for j in i]
seam2 = [j for i in seam2 for j in i]
seam3 = [j for i in seam3 for j in i]
seam4 = [j for i in seam4 for j in i]

s1 = hippo.copy()
s1.visual.vertex_colors[seam1] = [242, 255, 94, 1]
s1.export("g_meshes/E7/low.obj")

s2 = hippo.copy()
s2.visual.vertex_colors[seam2] = [242, 255, 94, 1]
s2.export("g_meshes/E7/lowmid.obj")

s3 = hippo.copy()
s3.visual.vertex_colors[seam3] = [242, 255, 94, 1]
s3.export("g_meshes/E7/mid.obj")

s4 = hippo.copy()
s4.visual.vertex_colors[seam4] = [242, 255, 94, 1]
s4.export("g_meshes/E7/high.obj")

c1 = delete_faces_along_seam(hippo, length_divisor = 0.01, threshold = 0.2)
c2 = delete_faces_along_seam(hippo, length_divisor = 0.5, threshold = 0.2)
c3 = delete_faces_along_seam(hippo, length_divisor = 1, threshold = 0.2)
c4 = delete_faces_along_seam(hippo, length_divisor = 1.5, threshold = 0.2)

c1.export("g_meshes/E7/b_low.obj")
c2.export("g_meshes/E7/b_lowmid.obj")
c3.export("g_meshes/E7/b_mid.obj")
c4.export("g_meshes/E7/b_high.obj")

export_textured("g_meshes/E7/b_low.obj", "g_meshes/E7/tex_b_low.obj", "g_meshes/flat1.obj")
export_textured("g_meshes/E7/b_lowmid.obj", "g_meshes/E7/tex_b_lowmid.obj", "g_meshes/flat1.obj")
export_textured("g_meshes/E7/b_mid.obj", "g_meshes/E7/tex_b_mid.obj", "g_meshes/flat1.obj")
export_textured("g_meshes/E7/b_high.obj", "g_meshes/E7/tex_b_high.obj", "g_meshes/flat1.obj")


In [218]:
# Experiment 8 Number of points to use on the seam (percent to search through)

hippo = trimesh.load("closed_meshes/hippo.obj")

w1 = waypoints(hippo, percent = 1, threshold = 0.2)
w2 = waypoints(hippo, percent = 2, threshold = 0.2)
w3 = waypoints(hippo, percent = 3, threshold = 0.2)
w4 = waypoints(hippo, percent = 10, threshold = 0.2)

w1 = np.unique([j for i in w1 for j in i])
w2 = np.unique([j for i in w2 for j in i])
w3 = np.unique([j for i in w3 for j in i])
w4 = np.unique([j for i in w4 for j in i])

print(len(w1), len(w2), len(w3), len(w4))

seam1 = seam(hippo, length_divisor = 0.01, percent = 1, threshold = 0.2)
seam2 = seam(hippo, length_divisor = 0.01, percent = 2, threshold = 0.2)
seam3 = seam(hippo, length_divisor = 0.01, percent = 3, threshold = 0.2)
seam4 = seam(hippo, length_divisor = 0.01, percent = 10, threshold = 0.2)

seam1 = np.unique([j for i in seam1 for j in i])
seam2 = np.unique([j for i in seam2 for j in i])
seam3 = np.unique([j for i in seam3 for j in i])
seam4 = np.unique([j for i in seam4 for j in i])

s1 = hippo.copy()
s1.visual.vertex_colors[seam1] = [242, 255, 94, 1]
s1.export("g_meshes/E8/low.obj")

s2 = hippo.copy()
s2.visual.vertex_colors[seam2] = [242, 255, 94, 1]
s2.export("g_meshes/E8/lowmid.obj")

s3 = hippo.copy()
s3.visual.vertex_colors[seam3] = [242, 255, 94, 1]
s3.export("g_meshes/E8/mid.obj")

s4 = hippo.copy()
s4.visual.vertex_colors[seam4] = [242, 255, 94, 1]
s4.export("g_meshes/E8/high.obj")

c1 = delete_faces_along_seam(hippo, percent = 1, threshold = 0.2, length_divisor = 0.01)
c2 = delete_faces_along_seam(hippo, percent = 2, threshold = 0.2, length_divisor = 0.01)
c3 = delete_faces_along_seam(hippo, percent = 3, threshold = 0.2, length_divisor = 0.01)
c4 = delete_faces_along_seam(hippo, percent = 10, threshold = 0.2, length_divisor = 0.01)

c1.export("g_meshes/E8/b_low.obj")
c2.export("g_meshes/E8/b_lowmid.obj")
c3.export("g_meshes/E8/b_mid.obj")
c4.export("g_meshes/E8/b_high.obj")

export_textured("g_meshes/E8/b_low.obj", "g_meshes/E8/tex_b_low.obj", "g_meshes/flat1.obj")
export_textured("g_meshes/E8/b_lowmid.obj", "g_meshes/E8/tex_b_lowmid.obj", "g_meshes/flat1.obj")
export_textured("g_meshes/E8/b_mid.obj", "g_meshes/E8/tex_b_mid.obj", "g_meshes/flat1.obj")
export_textured("g_meshes/E8/b_high.obj", "g_meshes/E8/tex_b_high.obj", "g_meshes/flat1.obj")

6 17 26 53


KeyboardInterrupt: 

In [221]:
# Experiment 9 open vs closed

o_hand = trimesh.load("open_meshes/hand.obj")
c_hand = trimesh.load("closed_meshes/closed_hand.obj")

seam1 = seam(c_hand, length_divisor = 0.1, percent = 3, threshold = 2)
seam1 = np.unique([j for i in seam1 for j in i])

s = c_hand.copy()
s.visual.vertex_colors[seam1] = [242, 255, 94, 1]
s.export("g_meshes/E9/seam.obj")

c = delete_faces_along_seam(c_hand, length_divisor = 0.1, percent = 5, threshold = 2)
c.export("g_meshes/E9/cutouts.obj")
export_textured("g_meshes/E9/cutouts.obj", "g_meshes/E9/closed_hand.obj", "g_meshes/flat1.obj")

export_textured("open_meshes/hand.obj", "g_meshes/E9/open_hand.obj", "g_meshes/E9/flat1.obj", bound_func = circle_boundary)

o_horse = trimesh.load("open_meshes/open_horse_head.obj")
c_horse = trimesh.load("closed_meshes/closed_horse_head.obj")

seam2 = seam(c_horse, length_divisor = 0.01, percent = 2, threshold = 2)
seam2 = np.unique([j for i in seam2 for j in i])

s2 = c_horse.copy()
s2.visual.vertex_colors[seam2] = [242, 255, 94, 1]
s2.export("g_meshes/E9/seam2.obj")

c2 = delete_faces_along_seam(c_horse, length_divisor = 0.1, percent = 2, threshold = 2)
c2.export("g_meshes/E9/cutouts2.obj")
export_textured("g_meshes/E9/cutouts2.obj", "g_meshes/E9/closed_horse.obj", "g_meshes/flat1.obj")
export_textured("open_meshes/open_horse_head.obj", "g_meshes/E9/open_horse.obj", "g_meshes/E9/flat1.obj", bound_func = circle_boundary)